In [ ]:
# Publication Type Map	
# Working Paper	WP
# Journal Article	AJ
# Book Chapter	BC
# Book	BO
# Media Article	MA
# Policy Brief	PB
# Report	RPRT

In [2]:
import pandas as pd

In [212]:
excel_file_path = 'MASTER PUBS DATABASE.xlsx'

sheet_name = 'MASTER' 

master_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

In [213]:
master_df.columns

Index(['Author Salesforce Affiliation number', 'Type of Publication', 'Number',
       'Affiliate Author (First Name)', 'Affiliate Author (Middle)',
       'Affiliate Author (Last Name)', 'All Authors', 'Publication Title',
       'Journal or Publisher title', 'Forthcoming', 'Volume number',
       'Issue number', 'Page numbers', 'Quarter/Month',
       'Working paper number (if applicable)', 'Book title (if applicable)',
       'Year', 'Link', 'Note', 'Note2', 'Unsure/question', 'RB reviewed?',
       'Citation', 'Topic Tag', 'Region Tag', 'Countries', 'Program Tag',
       'Abstract', 'Sub-tags', 'When collected?', 'Column1', 'Page_Header'],
      dtype='object')

In [214]:
quarter_month_map = {
    '1st Quarter': '01',
    '2nd Quarter': '04',
    '3rd Quarter': '07',
    '4th Quarter': '10',
    'Fall': '09',
    'Winter': '12',
    'Spring': '03',
    'Autumn': '09',
    'Fall/Winter' : '09',
    'January': '01',
    'February': '02',
    'March': '03',
    'April': '04',
    'May': '05',
    'June': '06',
    'July': '07',
    'August': '08',
    'September': '09',
    'October': '10',
    'November': '11',
    'December': '12'
}

def map_quarter_month(val):
    return quarter_month_map.get(val, '')

def format_date(row):
    if 'Date' in row.index and pd.notnull(row['Date']):
        return row['Date']
    elif pd.notnull(row['Quarter/Month']):
        month_num = map_quarter_month(row['Quarter/Month'])
        if month_num:
            return f"{month_num}/01/{row['Year']}"
    else:
        return f"01/01/{row['Year']}"

# Apply the formatting logic to create a new column
master_df['Effective date'] = master_df.apply(format_date, axis=1)


In [215]:
master_df['Effective date'].head(10)

0    05/01/2023
1    05/01/2023
2    01/01/2016
3    03/01/2019
4    01/01/2020
5    01/01/2018
6    01/01/2014
7    01/01/2013
8    01/01/2009
9    04/01/2006
Name: Effective date, dtype: object

#### Editor Final

In [216]:
# import re

# def format_editors(editors):
#     # Split the editors using either ";" or ","
#     editor_list = re.split(r'[;,]', editors)
    
#     # Remove any leading or trailing whitespaces
#     editor_list = [editor.strip() for editor in editor_list if editor.strip()]
    
#     # If there are more than one editor, join all but the last one with ","
#     if len(editor_list) > 1:
#         formatted_editors = ", ".join(editor_list[:-1]) + " and " + editor_list[-1]
#     else:
#         formatted_editors = editor_list[0] if editor_list else ""
    
#     return formatted_editors

# # Apply the formatting function to the "Edited by (if applicable)" column
# master_df["Editors_Final"] = master_df["Edited by (if applicable)"].apply(format_editors)



In [217]:

def check_affiliate_author(row):
    if pd.notnull(row['Affiliate Author (Last Name)']):
        if isinstance(row['All Authors'], str):
            return 'Yes' if row['Affiliate Author (Last Name)'] in row['All Authors'] else 'No'
        elif isinstance(row['All Authors'], float) and pd.isna(row['All Authors']):
            return 'No'
    return ''

# Apply the function to create a new column
master_df['FA in All Authors'] = master_df.apply(check_affiliate_author, axis=1)

### Author Compilation   

In [218]:
master_df['All Authors'] 

0       Eric Priest; Celia Deane-Drummond; Joseph Henr...
1                                          Joseph Henrich
2                                             Gautam Nair
3                                Emily Blanchard; Mark Wu
4                                  Rodney Ludema; Mark Wu
                              ...                        
2175                        Xavier Gabaix; Thomas Graeber
2176                    Xavier Gabaix; Ralph S. J. Koijen
2177                                  Gabaix X; Laibson D
2178                                       Melani Cammett
2179                  Melani Cammett; Pauline Jones Luong
Name: All Authors, Length: 2180, dtype: object

In [219]:
import pandas as pd
import numpy as np
import re

def reverse_first_author_name(author_string):
    # Check if the value is NaN
    if pd.isna(author_string):
        return author_string
    
    authors = [author.strip() for author in re.split(';', author_string)]
    
    if len(authors) >= 1:
        first_author = authors[0].split()
        if len(first_author) >= 2:
            reversed_name = first_author[-1] + " " + " ".join(first_author[:-1])  # Change ' to "and"
            authors[0] = reversed_name
    
    author_string = ", ".join(authors)
    
    author_string = author_string.rsplit(', ', 1)
    author_string = ' and '.join(author_string)
    
    return author_string

master_df['All_Authors_CC'] = master_df['All Authors'].apply(reverse_first_author_name)


## Citiation   

In [220]:
import pandas as pd

#### VBA Functions

In [321]:
def wp_citation(row):
    authors = str(row["All_Authors_CC"])
    title = str(row["Publication Title"])
    working_paper_number = str(row["Working paper number (if applicable)"])
    publisher = str(row["Journal or Publisher title"])
    year = str(row["Year"])
    
    # if pd.isna(publisher):
    #     return "Check Mapping"
    # else:
    #     
    publisher = publisher
    # Construct the publication string
    publication_string = authors
    
    if len(title) > 0:
        if title[-1] in [".", "?"]:
            publication_string += ". " + title
        else:
            publication_string += ". " + title + "."
    
    if len(working_paper_number) > 0:
        publication_string += f" Working Paper no. {working_paper_number}."
    
    if len(publisher) > 0:
        publication_string += " " + publisher
    
    if len(str(year)) > 0:
        publication_string += f". {year}."
    else:
        publication_string += "."
    
    # Clean the publication string
    return publication_string


In [322]:
def book_citation(row):
    authors = str(row["All_Authors_CC"])
    title = str(row["Publication Title"])
    vol_number = str(row["Volume number"]) if pd.notna(row["Volume number"]) else ""
    publisher = str(row["Journal or Publisher title"])
    year = str(row["Year"]) if pd.notna(row["Year"]) else ""

    # place = row["Location (e.g.; city)"] if pd.notna(row["Location (e.g.; city)"]) else ""
    
    # if pd.isna(publisher):
    #     return "Check Mapping"
    
    publication_string = authors
    
    if len(str(year)) > 0:
        publication_string += f". {year}"
    
    if len(title) > 0:
        if title[-1] in [".", "?"]:
            publication_string += f". {title}"
        else:
            publication_string += f". {title}."
    
    if len(vol_number) > 0:
        publication_string += f" Vol. No.: {vol_number}."
    
    # if len(place) > 0:
    #     publication_string += f" {place}"
    
    if len(publisher) > 0:
        publication_string += f": {publisher}."
    else:
        publication_string += "."

    return publication_string.strip()


In [323]:
def JournalCitation(cell):
    authors = str(cell["All_Authors_CC"])
    title = str(cell["Publication Title"])
    volNumber = str(cell["Volume number"])
    issue = str(cell["Issue number"])
    publisher = str(cell["Journal or Publisher title"])
    year = str(cell["Year"])
    pageNumbers = str(cell["Page numbers"])
    pubMonth = str(cell["Quarter/Month"])

    
    # if isinstance(publisher, str):
    #     return "Check Mapping"
    # else:
    # publisher = str(publisher)  # Account for blank publications in Self-Published
    
    publicationString = authors
    
    if len(str(year)) > 0:
        publicationString += f". {year}"
    
    if len(title) > 0:
        publicationString += f". {title}"
    
    if len(publisher) > 0:
        if title and (title[-1] == "." or title[-1] == "?"):
            publicationString += f" {publisher}"
        else:
            publicationString += f". {publisher}"
    
    if len(volNumber) > 0:
        publicationString += f" {volNumber}"
    
    if len(issue) > 0:
        publicationString += f", no. {issue}"
    
    if len(pubMonth) > 0:
        publicationString += f" ({pubMonth})"
    
    if len(pageNumbers) > 0:
        publicationString += f": {pageNumbers}."
    else:
        publicationString += "."
    
    return publicationString

In [324]:
from datetime import datetime

def MediaCitation(cell):
    authors = str(cell["All_Authors_CC"])
    title = str(cell["Publication Title"])
    datePub = str(cell["Effective date"])
    publisher = str(cell["Journal or Publisher title"])
    year = str(cell["Year"])
    pageNumbers = str(cell["Page numbers"])

    # Convert publisher to string and check for error
    # if isinstance(publisher, str):
    #     return "Check Mapping"
    # else:
    #     publisher = str(publisher)  # Account for blank publications in Self-Published
    
    # Construct the publication string
    publicationString = authors
    
    if len(str(year)) > 0:
        publicationString += f". {year}. "
    
    if len(title) > 0:
        if title[-1] == "." or title[-1] == "?":
            publicationString += f"{title} "
        else:
            publicationString += f"{title}. "
    
    if len(publisher) > 0:
        publicationString += publisher
    
    if isinstance(datePub, datetime):
        publicationString += f", {datePub.strftime('%B %d, %Y')}"
    
    if len(pageNumbers) > 0:
        publicationString += f", {pageNumbers}."
    else:
        publicationString += "."
    
    # Clean the publication string and return
    return publicationString


In [325]:
def BookChapterCitation(cell):
    authors = str(cell["All_Authors_CC"])
    chap_title = str(cell["Publication Title"])
    volNumber = str(cell["Volume number"])
    # editor = str(cell["Editors_Final"]).strip()
    book_title = str(cell["Book title (if applicable)"])
    authorFlag = str(cell["FA in All Authors"])

    # Check if publisher is causing an error and if yes return "Check Mapping" warning
    publisher = cell["Journal or Publisher title"]
    # if isinstance(publisher, str):
    #     return "Check Mapping"
    # else:
    #     publisher = str(publisher)  # Account for blank publications in Self-Published
    
    year = cell["Year"]
    # place = cell["Location (e.g.; city)"] if pd.notna(cell["Location (e.g.; city)"]) else ""
    
    # Construct the publication string
    # Adding conditional statements that remove blank fields
    # Check if FA is in all authors to proceed with the citation
    
    if authorFlag == "No":
        return "Issue in Author Name"
    else:
        publicationString = authors
        
        if len(str(year)) > 0:
            publicationString += f". {year}"
        
        if len(chap_title) > 0:
            publicationString += f". \"{chap_title}\""
        
        if len(book_title) > 0:
            publicationString += f" in {book_title}"
        
        # if len(editor) > 0:
        #     publicationString += f", edited by {editor}"
            
        if len(volNumber) > 0:
            publicationString += f". Vol. No.: {volNumber}"
        
        # if len(place) > 0:
        #     publicationString += f". {place}"
        
        if len(publisher) > 0:
            publicationString += f": {publisher}."
        else:
            publicationString += "."
        
        # Display the result
        return publicationString.strip()


In [326]:
def ReportCitation(cell):
    authors = str(cell["Authors_Final"])
    title = str(cell["Publication Title"])
    
    # Check if publisher is causing an error and if yes return "Check Mapping" warning
    publisher = cell["Journal or Publisher (Publication Series Title)"]
    # if isinstance(publisher, str):
    #     return "Check Mapping"
    # else:
    publisher = str(publisher) # Account for blank publications in Self-Published
    
    year = cell.ListObject.ListColumns("Year").DataBodyRange.Cells(cell.Row - 1).Value.strip()
    
    # Construct the publication string
    # Adding conditional statements that remove blank fields
    # publicationString = [author name(s)]. [report title]. [publisher], [year].
    
    publicationString = authors
    
    if len(title) > 0:
        publicationString += f". {title}"
    
    if len(publisher) > 0:
        if title[-1] == "." or title[-1] == "?":
            publicationString += f" {publisher}"
        else:
            publicationString += f". {publisher}"
    
    if len(str(year)) > 0:
        publicationString += f", {year}"
    
    publicationString += "."
    
    # Display the result
    return publicationString.strip()


#### VBA Functions

In [327]:
def citation_generator_2(master_df):
    for index, row in master_df.iterrows():
        pubTypeCd = row["Type of Publication"]
        
        if pubTypeCd == "Working Paper":
            citation = wp_citation(row)
        elif pubTypeCd == "Book":
            citation = book_citation(row)
        elif pubTypeCd == "Journal Article":
            citation = JournalCitation(row)
        elif pubTypeCd == "Media Article":
            citation = MediaCitation(row)
        elif pubTypeCd == "Book Chapter":
            citation = BookChapterCitation(row)
        elif pubTypeCd == "Report":
            citation = ReportCitation(row)
        else:
            citation = "Skipped"
        
        master_df.at[index, "Citation_Final"] = citation
    
    print("Citations Completed")

citation_generator_2(master_df)
master_df[['Type of Publication', 'Citation_Final']].head(50)

Citations Completed


,Type of Publication,Citation_Final
0,Journal Article,"Priest Eric, Celia Deane-Drummond, Joseph Henr..."
1,Journal Article,Henrich Joseph. 2023. How Culture Made Us Uniq...
2,Media Article,Nair Gautam. 2016. Why the Poor Don’t Soak the...
3,Journal Article,Blanchard Emily and Mark Wu. 2019. Externaliti...
4,Journal Article,Ludema Rodney and Mark Wu. 2020. What is Price...
5,Book,Melitz Marc J.. 2018. Trade Competition and Re...
6,Book,Friedman Benjamin M.. 2014. Is Our Economy's F...
7,Book,Friedman Benjamin M.. 2013. Is Our Economy's F...
8,Book Chapters,Skipped
9,Journal Article,"Alsan Marcella, David E Bloom and David Cannin..."


In [335]:
master_df

,Author Salesforce Affiliation number,Type of Publication,Number,Affiliate Author (First Name),Affiliate Author (Middle),Affiliate Author (Last Name),All Authors,Publication Title,Journal or Publisher title,Forthcoming,...,Program Tag,Abstract,Sub-tags,When collected?,Column1,Page_Header,Effective date,FA in All Authors,All_Authors_CC,Citation_Final
0,NaN,Journal Article,56,Joseph,NaN,Henrich,Eric Priest; Celia Deane-Drummond; Joseph Henr...,Introduction To Symposium On “Just How Special...,Zygon,No,...,NaN,NaN,NaN,Summer - Kiki,NaN,abstract-text,05/01/2023,Yes,"Priest Eric, Celia Deane-Drummond, Joseph Henr...","Priest Eric, Celia Deane-Drummond, Joseph Henr..."
1,NaN,Journal Article,56,Joseph,NaN,Henrich,Joseph Henrich,How Culture Made Us Uniquely Human,Zygon,No,...,NaN,NaN,NaN,Summer - Kiki,NaN,NaN,05/01/2023,Yes,Henrich Joseph,Henrich Joseph. 2023. How Culture Made Us Uniq...
2,NaN,Media Article,27,Gautam,NaN,Nair,Gautam Nair,Why the Poor Don’t Soak the Rich: They Don’t K...,Yale Institution for Social and Policy Studies...,No,...,NaN,NaN,NaN,Summer - Kiki,NaN,NaN,01/01/2016,Yes,Nair Gautam,Nair Gautam. 2016. Why the Poor Don’t Soak the...
3,NaN,Journal Article,111,Mark,NaN,Wu,Emily Blanchard; Mark Wu,Externalities and Agricultural Import Bans: Ev...,World Trade Review,NaN,...,NaN,NaN,NaN,Summer - Maceo,NaN,NaN,03/01/2019,Yes,Blanchard Emily and Mark Wu,Blanchard Emily and Mark Wu. 2019. Externaliti...
4,NaN,Journal Article,111,Mark,NaN,Wu,Rodney Ludema; Mark Wu,What is Price Suppression in Abnormal Economic...,World Trade Rev.,NaN,...,NaN,NaN,NaN,Summer - Maceo,NaN,NaN,01/01/2020,Yes,Ludema Rodney and Mark Wu,Ludema Rodney and Mark Wu. 2020. What is Price...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2175,NaN,Working Paper,127,Xavier,NaN,Gabaix,Xavier Gabaix; Thomas Graeber,The Complexity of Economic Decisions,NaN,NaN,...,NaN,NaN,NaN,Summer - Maceo,NaN,abstract-text,01/01/2023,Yes,Gabaix Xavier and Thomas Graeber,Gabaix Xavier and Thomas Graeber. The Complexi...
2176,NaN,Working Paper,127,Xavier,NaN,Gabaix,Xavier Gabaix; Ralph S. J. Koijen,Granular Instrumental Variables,NaN,NaN,...,NaN,NaN,NaN,Summer - Maceo,NaN,abstract-text,04/01/2023,Yes,Gabaix Xavier and Ralph S. J. Koijen,Gabaix Xavier and Ralph S. J. Koijen. Granular...
2177,NaN,Working Paper,127,Xavier,NaN,Gabaix,Gabaix X; Laibson D,Myopia and Discounting,NaN,NaN,...,NaN,NaN,NaN,Summer - Maceo,NaN,NaN,01/01/2022,Yes,X Gabaix and Laibson D,X Gabaix and Laibson D. Myopia and Discounting...
2178,NaN,Media Article,85,Melani,NaN,Cammett,Melani Cammett,The Syrian Conflict’s Impact on Lebanese Politics,NaN,No,...,NaN,NaN,NaN,Kiki - 10/8/23 batch,NaN,NaN,11/01/2013,Yes,Cammett Melani,Cammett Melani. 2013. The Syrian Conflict’s Im...


In [336]:
master_df.to_csv('output.csv', index=False)


In [330]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
from thefuzz import fuzz


In [331]:

# Define a function for fuzzy matching
def fuzz_match(row, col_to_check, ref_table, ref_col, output_format='tuple'):
    '''
    Perform fuzzy matching between defined column and reference column
    Arguments:
    - row (pd.Series): The row containing the value to match.
    - col_to_check (str): The name of the column containing the value to match.
    - ref_table (pd.Series or pd.DataFrame): The reference table for matching.
    - ref_col (str): Name of the column containing the reference values 
    - output_format (str, optional): The desired output format.
        - 'tuple': Return a tuple with (First Name, Middle Name, Last Name, Full Name). Use for name_cleaning
        - 'closest_match': Return the closest match along with the fuzzy ratio. Use for publisher_cleaning
        
    '''
    ref_name = row[col_to_check]
    sim_score = ref_table[ref_col].apply(lambda x: fuzz.ratio(ref_name, x))
    
    if any(sim_score == 100):
        if output_format == 'tuple':
            return (None, None, None, None)
        elif output_format == 'closest_match':
            perfect_match_idx = sim_score[sim_score == 100].index[0]
            return ref_table.loc[perfect_match_idx, ref_col], 100
    else:
        closest_match_idx = sim_score.idxmax()
       
        if output_format == 'tuple':
            corr_values = ref_table.loc[closest_match_idx]
            corr_values = corr_values.apply(lambda x: None if pd.isna(x) else x)
        elif output_format == 'closest_match':
            corr_values = ref_table.loc[closest_match_idx, ref_col]
            
        if output_format == 'tuple':
            return tuple(corr_values)
        elif output_format == 'closest_match':
            return corr_values, sim_score.max()


In [332]:
# # Read raw data and publisher mapping file
# raw_data = master_df.copy()
# publisher_map = pd.read_csv("E:/Harvard/CID Pubs Database/Python Version/Publisher_mapping.csv")

In [333]:

# Rename columns in the raw data
col_names = ["sf_affln_num", "pub_id", "pub_type", "fa_num", "fa_school", "fa_firstname", "fa_midname", "fa_lastname",
             "all_auth", "editors", "pub_title", "subtitle", "publisher", "publisher_map", "frthcoming_flag", 
             "vol_num", "issue_num", "pg_num", "qtr_mnth", "wp_num", "book_title", "place", "year", "date", 
             "month", "date_eff", "url"]
raw_data.columns = col_names + list(raw_data.columns[len(col_names):])


In [334]:

# Clean publisher names
raw_data["publisher_map"] = raw_data["publisher"].str.strip()
publisher_raw = publisher_map.merge(raw_data["publisher_map"], how="right", left_on="Original", right_on="publisher_map")
unmatched_pubs = publisher_raw.loc[publisher_raw["Mapped_name"].isna() 
                                   & ~publisher_raw["publisher_map"].isna() 
                                   & ~publisher_raw["publisher_map"].fillna('').str.contains("Self published") 
                                  ].drop_duplicates()
unmatched_pubs["publisher_lower"] = unmatched_pubs["publisher_map"].str.lower()
unmatched_pubs = unmatched_pubs.merge(publisher_map, how="left", left_on="publisher_lower", 
                                      right_on=publisher_map["Original"].str.lower())
matched_pubs_casecheck = unmatched_pubs.loc[~unmatched_pubs["Mapped_name_y"].isna(), 
                                            ["Original_y", "Mapped_name_y", "publisher_map"]]
matched_pubs_casecheck["Original_y"] = matched_pubs_casecheck["publisher_map"]
matched_pubs_casecheck.drop(columns="publisher_map", inplace=True)
matched_pubs_casecheck.columns = publisher_map.columns
publisher_map2 = pd.concat([publisher_map, matched_pubs_casecheck], ignore_index=True)

unmatched_pubs2 = unmatched_pubs.loc[unmatched_pubs["Mapped_name_y"].isna(), ["publisher_map", "Mapped_name_y"]]
result = unmatched_pubs2.apply(
    lambda row: fuzz_match(
        row,
        col_to_check="publisher_map",
        ref_table=publisher_map2,
        ref_col="Mapped_name",
        output_format="closest_match"
    ),
    axis=1, result_type="expand"
)
unmatched_pubs2["Mapped_name_y"] = result[0]
unmatched_pubs2["Match_Score"] = result[1]
matched_pubs_fuzz = unmatched_pubs2.loc[unmatched_pubs2["Match_Score"] > 90,
                                        ["publisher_map", "Mapped_name_y"]]
matched_pubs_fuzz.columns = publisher_map2.columns
publication_map2 = pd.concat([publisher_map2, matched_pubs_fuzz], ignore_index=True)


NameError: name 'publisher_map' is not defined